# Setup

In [15]:
!pip install datasets transformers torch accelerate sacremoses sacrebleu --quiet

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [16]:
from datasets import load_dataset, load_metric
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import numpy as np
import torch, os

## Load the dataset

In [17]:
raw_dataset = load_dataset('ai4bharat/samanantar', 'hi', split='train', streaming=True, trust_remote_code=True)

In [18]:
# Take the first 200,000 rows
limited_data = raw_dataset.take(200000)

In [19]:
limited_data

IterableDataset({
    features: ['idx', 'src', 'tgt'],
    n_shards: 1
})

In [20]:
# Convert the IterableDataset to a list
limited_data_list = list(limited_data)

# Create a Dataset from the list
limited_data = Dataset.from_list(limited_data_list)

# Create a DatasetDict
dataset_dict = DatasetDict({"train": limited_data})

# Verify the first example to ensure conversion was successful
print(dataset_dict["train"][0])


{'idx': 0, 'src': "However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles", 'tgt': 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।'}


In [21]:
# Convert the dataset to a Pandas DataFrame
train_df = dataset_dict["train"].to_pandas()

# Rename the columns
train_df = train_df.rename(columns={"src": "en", "tgt": "hi"})

# Drop the 'idx' column if it is not needed
train_df = train_df.drop(columns=["idx"])

# Display the first few rows to verify
train_df.head()


,en,hi
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,The value of insects in the biosphere is enorm...,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ..."
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व..."


## Prepare the dataset for the model

In [22]:
train_df, val_df = train_test_split(train_df, test_size=0.1)

# Create Hugging Face Datasets from the DataFrames
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'hi', '__index_level_0__'],
        num_rows: 180000
    })
    validation: Dataset({
        features: ['en', 'hi', '__index_level_0__'],
        num_rows: 20000
    })
})

# Load the Model and Tokenizer

In [24]:
model_name = 'Helsinki-NLP/opus-mt-en-mul'
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Define a function to tokenize the dataset
def tokenize_function(examples):
    inputs = examples['en']
    targets = examples['hi']
    model_inputs = tokenizer(inputs, text_target=targets, truncation=True, padding="max_length", max_length=128)
    return model_inputs

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/180000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'hi', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 180000
    })
    validation: Dataset({
        features: ['en', 'hi', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"

In [27]:
# Load the model
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    warmup_steps=500,  
    gradient_accumulation_steps=2,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True
)

# Define the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define a function to compute metrics
metric = load_metric('sacrebleu', trust_remote_code=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU score
    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result

# Create the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [28]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,0.666300,0.602311,14.266666,45.135550
1,0.620900,0.578757,15.235178,45.047350


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]], 'forced_eos_token_id': 0}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

TrainOutput(global_step=5624, training_loss=0.7583249880273855, metrics={'train_runtime': 10677.0962, 'train_samples_per_second': 33.717, 'train_steps_per_second': 0.527, 'total_flos': 1.2201241878724608e+16, 'train_loss': 0.7583249880273855, 'epoch': 1.9996444444444443})

# Save the Finetuned Model and Tokenizer

In [29]:
# Save directory
save_directory = './finetuned-opusmt-en-to-hi'

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]], 'forced_eos_token_id': 0}


Model saved to ./finetuned-opusmt-en-to-hi


# Load the finetuned model and check out the translation

In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your model and tokenizer
model_name =save_directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example English sentence to translate
english_sentence = "My name is Varsha"

# Tokenize the input sentence
inputs = tokenizer(english_sentence, return_tensors="pt").to(device)

# Generate translation
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Decode the generated tokens
translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"English: {english_sentence}")
print(f"Hindi Translation: {translated_sentence}")


English: My name is Varsha
Hindi Translation: मेरा नाम वर्षा है
